In [1]:
import tensorflow 
import numpy as np 
import pandas as pd 
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio
import music21
from music21 import *
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adamax
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import sys
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
np.random.seed(42)

In [15]:
from music21 import converter
import os

filepath = "./albeniz/"
# Getting midi files
all_midis = []
for i in os.listdir(filepath):
    if i.endswith(".mid"):
        tr = os.path.join(filepath, i)  # Ensures the correct path joining
        midi = converter.parse(tr)
        all_midis.append(midi)


Use a library like music21 to extract notes and chords from the MIDI files

In [22]:
from music21 import converter, instrument, note, chord

notes = []
for midi_file in all_midis:
    parts = midi_file.getElementsByClass('Part')  # Use 'Part' in quotes to fetch the class
    for part in parts:
        elements = part.recurse()  # Recursively get all elements
        for element in elements:
            if isinstance(element, note.Note):  # If element is a Note
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):  # If element is a Chord
                # Join normalOrder to create a representation of the chord
                notes.append('.'.join(str(n) for n in element.normalOrder))



Convert the sequences of notes/chords into input/output pairs for training the LSTM.


In [25]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical  # Correct import

# Encode the notes
label_encoder = LabelEncoder()
int_notes = label_encoder.fit_transform(notes)

sequence_length = 100
input_sequences = []
output_sequences = []

for i in range(len(int_notes) - sequence_length):
    input_sequences.append(int_notes[i:i + sequence_length])
    output_sequences.append(int_notes[i + sequence_length])

# Reshape and normalize
input_sequences = np.reshape(input_sequences, (len(input_sequences), sequence_length, 1))
input_sequences = input_sequences / float(len(set(int_notes)))

# One-hot encode the output sequences
output_sequences = to_categorical(output_sequences)  # Correct function name


Create an LSTM model using Keras or TensorFlow

In [29]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.callbacks import EarlyStopping

# Define early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',   # Metric to monitor
    patience=10,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore the weights of the best model after stopping
)



model = Sequential()
model.add(LSTM(256, input_shape=(input_sequences.shape[1], input_sequences.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(len(set(notes))))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

validation_split = 0.2

In [30]:
history = model.fit(
    input_sequences, output_sequences,
    epochs=100,
    batch_size=64,
    validation_split=validation_split,  # Specify the percentage of data for validation
    callbacks=[early_stopping]          # Include the early stopping callback
)


Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 46s 195ms/step - loss: 4.8000 - val_loss: 4.9174
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 43s 197ms/step - loss: 4.4224 - val_loss: 5.0908
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 44s 199ms/step - loss: 4.3209 - val_loss: 5.3505
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 43s 198ms/step - loss: 4.2078 - val_loss: 5.2016
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 46s 208ms/step - loss: 4.1633 - val_loss: 5.2825
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 49s 224ms/step - loss: 4.0182 - val_loss: 5.2803
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 49s 224ms/step - loss: 3.9299 - val_loss: 5.3948
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 46s 210ms/step - loss: 3.7964 - val_loss: 5.4263
Epoch 9/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 45s 204ms/step - loss: 3.6625 - val_loss: 5.6374
Epoch 10/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 45s 204ms/step - loss: 3.4832 - val_loss: 5.7484
Epoch 11/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 45s 207ms/step - loss: 3.3388 - val_loss: 5.7552


In [32]:
import random
import numpy as np

# Randomly select a starting index
start_index = random.randint(0, len(input_sequences) - 1)
seed = input_sequences[start_index]

generated_notes = []

for i in range(500):  # Length of the generated music
    prediction = model.predict(seed, verbose=0)
    index = np.argmax(prediction)

    # Ensure the index is within the valid range of labels
    if index >= len(label_encoder.classes_):
        index = len(label_encoder.classes_) - 1  # Clip to the max valid index

    predicted_note = label_encoder.inverse_transform([index])[0]
    generated_notes.append(predicted_note)

    # Update the seed with the predicted note, keeping the sequence length constant
    seed = np.append(seed[1:], [[index]], axis=0)


In [33]:
from music21 import stream, note, chord, instrument

def create_midi(predicted_notes, output_file="output.mid"):
    # Create a music21 stream object
    midi_stream = stream.Stream()

    # Add an instrument part (optional)
    midi_stream.append(instrument.Piano())  # You can choose different instruments

    # Loop through the predicted sequence and convert to notes/chords
    for pattern in predicted_notes:
        # If the pattern is a chord (e.g., '60.64.67'), create a Chord
        if '.' in pattern or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes_in_chord = [int(n) for n in notes_in_chord]  # Convert to integers
            new_chord = chord.Chord(notes_in_chord)
            new_chord.duration.quarterLength = 0.5  # Set duration (can be adjusted)
            midi_stream.append(new_chord)
        else:  # If the pattern is a single note, create a Note
            new_note = note.Note(pattern)
            new_note.duration.quarterLength = 0.5  # Set duration (can be adjusted)
            midi_stream.append(new_note)

    # Write the stream to a MIDI file
    midi_stream.write('midi', fp=output_file)

# Example usage (replace 'generated_notes' with your actual predicted notes sequence)
create_midi(predicted_notes=generated_notes, output_file="generated_music.mid")
